In [ ]:
import ipywidgets as ipw
from aiidalab_widgets_base import CodQueryWidget, SmilesWidget, StructureExamplesWidget
from aiidalab_widgets_base import StructureBrowserWidget, StructureManagerWidget, StructureUploadWidget
#from aiidalab_optimade import OptimadeQueryWidget

from wizard import WizardApp
from structures import StructureSelectionStep
from bands import ComputeBandsSubmitWidget

from aiida.orm import StructureData


# Create the application steps
structure_manager_widget = StructureManagerWidget(
    importers=[
        StructureUploadWidget(title="From computer"),
        CodQueryWidget(title="COD"),
        StructureBrowserWidget(title="AiiDA database"),
#        OptimadeQueryWidget(title="OPTiMaDe"),
        SmilesWidget(title="SMILES"),  # requires OpenBabel! 
        StructureExamplesWidget(
            title="From Examples",
            examples=[
                ("Silicon", "miscellaneous/structures/Si.xyz"),
                ("Silicon oxide", "miscellaneous/structures/SiO2.xyz"),
            ]),
    ],
    node_class='StructureData',
)
structure_selection_step = StructureSelectionStep(manager=structure_manager_widget)
structure_selection_step.auto_next = True
compute_bands_step = ComputeBandsSubmitWidget()

# Link the application steps
ipw.dlink((structure_selection_step, 'confirmed_structure'), (compute_bands_step, 'input_structure'))


# Add the application steps to the application
app = WizardApp(
    steps=[
        ('Select structure', structure_selection_step),
        ('Compute bands', compute_bands_step)])
compute_bands_step.callbacks.append(lambda _: app._update_titles())

display(app)